<a href="https://colab.research.google.com/github/ninawang4/mbit-classifier/blob/main/Embeddings_XGBoost_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import pandas as pd

with open('/content/drive/MyDrive/NLP_project/Data/no_hashtags_mentions.csv', 'r', encoding='utf-8') as f:
    df = pd.DataFrame(csv.reader(f))
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    print(df.head())

0                                            caption    username  mbti  \
0  '번지점프를 하다' 작품으로 뮤지컬 데뷔할 수 있게 되어서 정말 영광이었습니다🥹 배...  suye.on2di  ISTJ   
1  꽃과 달⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀...  suye.on2di  ISTJ   
2  One day 가이드! 너무 행복한 작업이었다고 한다..🎇 작가님들 고생많으셨습니다...  suye.on2di  ISTJ   
3  안녕하세요 위키미키 지수연입니다. 멤버 유정이의 지목으로 의미 있는 챌린지에 함께 ...  suye.on2di  ISTJ   
4  2021년 소중한 사람들과 함께해서 증말 행복했어요☺️🥰 내년에도 잘 부탁해요🙌 모...  _haerimida  ESTP   

0      group    name caption_lengths extraversion sensing thinking judging  
0  Weki Meki  Suyeon             141            0       1        1       1  
1  Weki Meki  Suyeon              65            0       1        1       1  
2  Weki Meki  Suyeon              58            0       1        1       1  
3  Weki Meki  Suyeon             285            0       1        1       1  
4  Weki Meki    Elly              80            1       1        1       0  


In [ ]:
df.head()

,caption,username,mbti,group,name,caption_lengths,extraversion,sensing,thinking,judging
0,'번지점프를 하다' 작품으로 뮤지컬 데뷔할 수 있게 되어서 정말 영광이었습니다🥹 배...,suye.on2di,ISTJ,Weki Meki,Suyeon,141,0,1,1,1
1,꽃과 달⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀...,suye.on2di,ISTJ,Weki Meki,Suyeon,65,0,1,1,1
2,One day 가이드! 너무 행복한 작업이었다고 한다..🎇 작가님들 고생많으셨습니다...,suye.on2di,ISTJ,Weki Meki,Suyeon,58,0,1,1,1
3,안녕하세요 위키미키 지수연입니다. 멤버 유정이의 지목으로 의미 있는 챌린지에 함께 ...,suye.on2di,ISTJ,Weki Meki,Suyeon,285,0,1,1,1
4,2021년 소중한 사람들과 함께해서 증말 행복했어요☺️🥰 내년에도 잘 부탁해요🙌 모...,_haerimida,ESTP,Weki Meki,Elly,80,1,1,1,0


In [ ]:
import sys
# sys.path.append("/content/hw3")

import csv
import itertools as it
import numpy as np
import torch
import sklearn.decomposition
np.random.seed(0)
from tqdm import tqdm



In [ ]:
# from transformers import AutoTokenizer
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)


In [ ]:
from transformers import DistilBertModel

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
model.to(device)

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [ ]:
# embeddings = []
# attention_msks = []
# multiply = []

# for i, sent in enumerate(df['caption']):

#   # Encode the sentence
#   encoded = tokenizer.encode_plus(
#       text=sent,  # the sentence to be encoded
#       add_special_tokens=True,  # Add [CLS] and [SEP]
#       max_length = 512,  # maximum length of a sentence
#       pad_to_max_length=True,  # Add [PAD]s
#       return_attention_mask = True,  # Generate the attention mask
#       return_tensors = 'pt',  # ask the function to return PyTorch tensors
#   )

#   # Get the input IDs and attention mask in tensor format
#   input_ids = encoded['input_ids']
#   attn_mask = encoded['attention_mask']

#   embeddings.extend(input_ids)
#   attention_msks.extend(attn_mask)

#   # encoded_input = tokenizer(sent, return_tensors='tf')
#   # output_embeddings = model(encoded_input)
#   # embeddings.extend(output_embeddings)




In [ ]:
from torch.utils.data import TensorDataset
#tokenizer function from pset 3, with no labels involved (bc seq2seq)

def tokenizer_function(input_data):
  input_ids = []
  attention_masks = []
  for sent in input_data:
    this_encoding = tokenizer.encode_plus(sent, truncation=True, pad_to_max_length = True,max_length = 512,return_attention_mask = True,return_tensors = 'pt')
    input_ids.append(this_encoding['input_ids'])
    attention_masks.append( this_encoding['attention_mask'])
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  tokenized_data = TensorDataset(input_ids, attention_masks)
  return attention_masks, tokenized_data

In [ ]:
attn_msks, dataset = tokenizer_function(df['caption'].to_list())

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=False)  #no shuffling to keep embeddings aligned with the df
seq_lengths = torch.sum(attn_msks, dim=1).numpy()

In [ ]:
embedding_lst = []
i = 0

for batch in loader:
  if i % 50 == 0: print(i)

  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)

  with torch.no_grad():
      outputs = model(b_input_ids,
                      attention_mask=b_input_mask)

  states = outputs.last_hidden_state.detach().cpu().numpy() #(batch x seq_len x output_dim)
  embedding_avg = (states * b_input_mask.cpu().numpy()[:, :, np.newaxis]).sum(axis=1) / seq_lengths[32*i: 32*(i+1), np.newaxis] #compute average embedding of output -> (batch x output_dim)
  embedding_lst.extend(embedding_avg)

  i += 1

embeddings = np.stack(embedding_lst, axis=0)  #(num_data x seq_len x output_dim)
del embedding_lst #memory concerns

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250


## Dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 12)  #change this for num. dimensions
embeddings_pca = pca.fit_transform(embeddings)

In [ ]:
embedding_df = pd.DataFrame(embeddings_pca)
new_df = pd.concat([df, embedding_df], axis=1)

In [ ]:
import torch
from torch.utils.data import TensorDataset

mbti_dict = {'INTJ': 0, 'INTP': 1, 'ENTJ': 2, 'ENTP': 3, \
             'INFJ': 4, 'INFP': 5, 'ENFJ': 6, 'ENFP':7, \
             'ISTJ': 8, 'ISFJ': 9, 'ESTJ': 10, 'ESFJ': 11, \
             'ISTP': 12, 'ISFP': 13, 'ESTP': 14, 'ESFP': 15}

new_df['label'] = [mbti_dict[new_df['mbti'][i]] for i in range(new_df.shape[0])]
print(new_df.head())

                                             caption    username  mbti  \
0  '번지점프를 하다' 작품으로 뮤지컬 데뷔할 수 있게 되어서 정말 영광이었습니다🥹 배...  suye.on2di  ISTJ   
1  꽃과 달⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀...  suye.on2di  ISTJ   
2  One day 가이드! 너무 행복한 작업이었다고 한다..🎇 작가님들 고생많으셨습니다...  suye.on2di  ISTJ   
3  안녕하세요 위키미키 지수연입니다. 멤버 유정이의 지목으로 의미 있는 챌린지에 함께 ...  suye.on2di  ISTJ   
4  2021년 소중한 사람들과 함께해서 증말 행복했어요☺️🥰 내년에도 잘 부탁해요🙌 모...  _haerimida  ESTP   

       group    name caption_lengths extraversion sensing thinking judging  \
0  Weki Meki  Suyeon             141            0       1        1       1   
1  Weki Meki  Suyeon              65            0       1        1       1   
2  Weki Meki  Suyeon              58            0       1        1       1   
3  Weki Meki  Suyeon             285            0       1        1       1   
4  Weki Meki    Elly              80            1       1        1       0   

   ...         3         4         5         6         7         8         9  \
0  ...

In [ ]:
new_df.to_csv('/content/drive/MyDrive/NLP_project/Data/embeddings-seq2seq-12-2.csv', index = False)

## Start with embedding data

In [ ]:
import csv
import pandas as pd

with open('/content/drive/MyDrive/NLP_project/Data/embeddings-seq2seq-12-2.csv', 'r', encoding='utf-8') as f:
    df = pd.DataFrame(csv.reader(f))
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    print(df.head())

0                                            caption    username  mbti  \
0  '번지점프를 하다' 작품으로 뮤지컬 데뷔할 수 있게 되어서 정말 영광이었습니다🥹 배...  suye.on2di  ISTJ   
1  꽃과 달⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀...  suye.on2di  ISTJ   
2  One day 가이드! 너무 행복한 작업이었다고 한다..🎇 작가님들 고생많으셨습니다...  suye.on2di  ISTJ   
3  안녕하세요 위키미키 지수연입니다. 멤버 유정이의 지목으로 의미 있는 챌린지에 함께 ...  suye.on2di  ISTJ   
4  2021년 소중한 사람들과 함께해서 증말 행복했어요☺️🥰 내년에도 잘 부탁해요🙌 모...  _haerimida  ESTP   

0      group    name caption_lengths extraversion sensing thinking judging  \
0  Weki Meki  Suyeon             141            0       1        1       1   
1  Weki Meki  Suyeon              65            0       1        1       1   
2  Weki Meki  Suyeon              58            0       1        1       1   
3  Weki Meki  Suyeon             285            0       1        1       1   
4  Weki Meki    Elly              80            1       1        1       0   

0  ...                      3                     4                     5  \
0  ...  -

In [ ]:
for col in set(df.columns) - {'caption', 'embedding', 'username', 'mbti', 'group', 'name'}:
  df[col] = df[col].astype(float)

In [ ]:
# perform train-test-split by group, 0.80-0.20

from sklearn.model_selection import GroupShuffleSplit
np.random.seed(0)

# seems that this can only split into two groups, so we'll do it twice
train_splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=0)

# gets train indices, and (val + test) indices
train_split = train_splitter.split(df, groups=df['group'])
train_idx, test_idx = next(train_split)

train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]

In [ ]:
subsample = 1

X_train = train_df.loc[:, ~train_df.columns.isin(['caption', 'username', 'mbti', 'group', 'name', 'extraversion', 'sensing', 'thinking', 'judging', 'embedding', 'label', 'caption_lengths'])]
y_train = train_df['label'].astype(int)
X_test = test_df.loc[:, ~test_df.columns.isin(['caption', 'username', 'mbti', 'group', 'name', 'extraversion', 'sensing', 'thinking', 'judging', 'embedding', 'label', 'caption_lengths'])]
y_test = test_df['label'].astype(int)

X_train = X_train[::subsample]
y_train = y_train[::subsample]
X_test = X_test[::subsample]
y_test = y_test[::subsample]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# subsample = 1

# X_train = X_train[::subsample]
# X_test = X_test[::subsample]
# y_train = y_train[::subsample]
# y_test = y_test[::subsample]

In [ ]:
import xgboost as xgb

# Create regression matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": 16}

model = xgb.train(dtrain=dtrain_clf, params=params,num_boost_round=10)
predictions = model.predict(dtest_clf)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

flat_accuracy(predictions, y_test)

0.14233891863879772

In [ ]:
predictions = model.predict(dtrain_clf)
flat_accuracy(predictions, y_train)

0.3666106665907344

In [ ]:
#save and load model
import pickle
file_name = '/content/drive/MyDrive/NLP_project/Models/xgb_mbti_seq2seq-12-2.pkl'

# save
pickle.dump(model, open(file_name, "wb"))

# load
# xgb_model_loaded = pickle.load(open(file_name, "rb"))

## Repeat model training for EI, SN, TF, JP

In [ ]:
def bin_accuracy(preds, labels):
    pred_flat = np.round(preds)
    labels_flat = labels
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
df.head()

,caption,username,mbti,group,name,caption_lengths,extraversion,sensing,thinking,judging,...,3,4,5,6,7,8,9,10,11,label
0,'번지점프를 하다' 작품으로 뮤지컬 데뷔할 수 있게 되어서 정말 영광이었습니다🥹 배...,suye.on2di,ISTJ,Weki Meki,Suyeon,141.0,0.0,1.0,1.0,1.0,...,-0.031882,-0.190255,-0.063131,0.674995,0.476299,0.147244,-0.163155,0.301651,-0.456939,8.0
1,꽃과 달⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀...,suye.on2di,ISTJ,Weki Meki,Suyeon,65.0,0.0,1.0,1.0,1.0,...,-0.243594,2.030659,-0.885150,0.882401,-1.183649,1.978317,1.630645,-0.700752,0.119992,8.0
2,One day 가이드! 너무 행복한 작업이었다고 한다..🎇 작가님들 고생많으셨습니다...,suye.on2di,ISTJ,Weki Meki,Suyeon,58.0,0.0,1.0,1.0,1.0,...,0.151260,-0.237824,0.269290,-0.726361,-0.060810,0.578726,0.358001,-0.012832,-0.361133,8.0
3,안녕하세요 위키미키 지수연입니다. 멤버 유정이의 지목으로 의미 있는 챌린지에 함께 ...,suye.on2di,ISTJ,Weki Meki,Suyeon,285.0,0.0,1.0,1.0,1.0,...,-0.543950,0.609027,0.595899,0.743415,0.807163,-0.607588,-0.064350,0.413869,0.386679,8.0
4,2021년 소중한 사람들과 함께해서 증말 행복했어요☺️🥰 내년에도 잘 부탁해요🙌 모...,_haerimida,ESTP,Weki Meki,Elly,80.0,1.0,1.0,1.0,0.0,...,1.034022,0.343060,-1.960683,0.304818,0.040311,0.008652,0.382473,0.298086,0.527488,14.0


In [ ]:
y_train = train_df['extraversion'].astype(int)
y_test = test_df['extraversion'].astype(int)

dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

params = {"objective": "binary:logistic", "tree_method": "hist"}

model = xgb.train(dtrain=dtrain_clf, params=params,num_boost_round=10)
predictions = model.predict(dtest_clf)
bin_accuracy(predictions, y_test)

0.547244774572465

In [ ]:
file_name = '/content/drive/MyDrive/NLP_project/Models/xgb_ei_seq2seq-12-2.pkl'
pickle.dump(model, open(file_name, "wb"))

In [ ]:
y_train = train_df['sensing'].astype(int)
y_test = test_df['sensing'].astype(int)

dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

params = {"objective": "binary:logistic", "tree_method": "hist"}

model = xgb.train(dtrain=dtrain_clf, params=params,num_boost_round=10)
predictions = model.predict(dtest_clf)
bin_accuracy(predictions, y_test)

0.6693729486958024

In [ ]:
file_name = '/content/drive/MyDrive/NLP_project/Models/xgb_sn_seq2seq-12-2.pkl'
pickle.dump(model, open(file_name, "wb"))

In [ ]:
y_train = train_df['thinking'].astype(int)
y_test = test_df['thinking'].astype(int)

dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

params = {"objective": "binary:logistic", "tree_method": "hist"}

model = xgb.train(dtrain=dtrain_clf, params=params,num_boost_round=10)
predictions = model.predict(dtest_clf)
bin_accuracy(predictions, y_test)

0.7215408533425463

In [ ]:
file_name = '/content/drive/MyDrive/NLP_project/Models/xgb_tf_seq2seq-12-2.pkl'
pickle.dump(model, open(file_name, "wb"))

In [ ]:
y_train = train_df['judging'].astype(int)
y_test = test_df['judging'].astype(int)

dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

params = {"objective": "binary:logistic", "tree_method": "hist"}

model = xgb.train(dtrain=dtrain_clf, params=params,num_boost_round=10)
predictions = model.predict(dtest_clf)
bin_accuracy(predictions, y_test)

0.5023320089825531

In [ ]:
file_name = '/content/drive/MyDrive/NLP_project/Models/xgb_jp_seq2seq-12-2.pkl'
pickle.dump(model, open(file_name, "wb"))

## Combining individual predictions

In [ ]:
model_ei = pickle.load(open('/content/drive/MyDrive/NLP_project/Models/xgb_ei_seq2seq-12-2.pkl', "rb"))
model_sn = pickle.load(open('/content/drive/MyDrive/NLP_project/Models/xgb_sn_seq2seq-12-2.pkl', "rb"))
model_tf = pickle.load(open('/content/drive/MyDrive/NLP_project/Models/xgb_tf_seq2seq-12-2.pkl', "rb"))
model_jp = pickle.load(open('/content/drive/MyDrive/NLP_project/Models/xgb_jp_seq2seq-12-2.pkl', "rb"))

In [ ]:
dtest_clf = xgb.DMatrix(X_test, test_df['extraversion'].astype(int), enable_categorical=True)

predictions_ei = model_ei.predict(dtest_clf)
predictions_sn = model_sn.predict(dtest_clf)
predictions_tf = model_tf.predict(dtest_clf)
predictions_jp = model_jp.predict(dtest_clf)

predictions_ei = np.where(predictions_ei > 0.5, 'E', 'I')
predictions_sn = np.where(predictions_sn > 0.5, 'S', 'N')
predictions_tf = np.where(predictions_tf > 0.5, 'T', 'F')
predictions_jp = np.where(predictions_jp > 0.5, 'J', 'P')

predictions_mbti = np.char.add(np.char.add(predictions_ei, predictions_sn), np.char.add(predictions_tf, predictions_jp))

In [ ]:
predictions_mbti_num = np.array([mbti_dict[mbti] for mbti in predictions_mbti])

In [ ]:
(predictions_mbti_num == test_df['label'].astype(int)).mean()

0.17015028502332008

In [ ]:
len(test_df['label'])

5789

In [ ]:
unique_values, counts = np.unique(predictions_mbti_num, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count / len(predictions_mbti_num)} times")

0 occurs 0.008464328899637243 times
1 occurs 0.0008637070305752289 times
2 occurs 0.009328035930212471 times
3 occurs 0.002245638279495595 times
4 occurs 0.2105717740542408 times
5 occurs 0.07445154603558472 times
6 occurs 0.5500086370703058 times
7 occurs 0.06425980307479703 times
9 occurs 0.018137847642079808 times
11 occurs 0.011400932803593021 times
12 occurs 0.00017274140611504577 times
13 occurs 0.0347210226291242 times
14 occurs 0.0010364484366902746 times
15 occurs 0.014337536707548799 times


In [ ]:
for label in range(16):
  print(label, sum(train_df['label']==label) / len(train_df))

0 0.06509296961758593
1 0.019391212733847776
2 0.0731797602437427
3 0.054813633645604945
4 0.08266180699906034
5 0.08485435234488453
6 0.1651812409237165
7 0.1201059255673567
8 0.03271733249807796
9 0.0591702497223725
10 0.0448190438224323
11 0.038810900082576386
12 0.030040718699279594
13 0.1082889603918107
14 0.0011389845952333494
15 0.01973290811241778


In [ ]:
dtest_clf = xgb.DMatrix(X_train, train_df['extraversion'].astype(int), enable_categorical=True)

predictions_ei = model_ei.predict(dtest_clf)
predictions_sn = model_sn.predict(dtest_clf)
predictions_tf = model_tf.predict(dtest_clf)
predictions_jp = model_jp.predict(dtest_clf)

predictions_ei = np.where(predictions_ei > 0.5, 'E', 'I')
predictions_sn = np.where(predictions_sn > 0.5, 'S', 'N')
predictions_tf = np.where(predictions_tf > 0.5, 'T', 'F')
predictions_jp = np.where(predictions_jp > 0.5, 'J', 'P')

predictions_mbti = np.char.add(np.char.add(predictions_ei, predictions_sn), np.char.add(predictions_tf, predictions_jp))

predictions_mbti_num = np.array([mbti_dict[mbti] for mbti in predictions_mbti])

(predictions_mbti_num == train_df['label'].astype(int)).mean()

0.24448304336683846

Results:

0.222142601218201 accuracy on 20% of data

0.268417658019206 accuracy on 100% of data